# Notebook to analysze and summarize soildgrids data by latitude

In [113]:
import geemap
import ee
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tqdm.notebook as tqdm
import geopandas as gpd
from shapely.geometry import Polygon

In [3]:
ee.Initialize()

### Functions

In [45]:
def extract_values_per_latitude(image, lat_s, lat_n, chunk_size, reducer=ee.Reducer.sum(), intersection=None):
    out_dfs = []
    for i in tqdm.tqdm_notebook(np.arange(lat_s, lat_n, chunk_size)):
        try:
            # setup geoms
            latRange = ee.Geometry.BBox(-180+0.001, int(i), 180-0.001, int(i+chunk_size))
            fishnet = geemap.fishnet(latRange, h_interval=360, v_interval=1, intersect=False)
            # reduce to grid cell - on server
            reduced = image.reduceRegions(collection=fishnet, reducer=reducer)
            # extract and save to df
            df_temp = geemap.ee_to_df(reduced)
            out_dfs.append(df_temp)
        except:
            print('Error, skipped', i)
    # merge and clean
    df = pd.concat(out_dfs)
    df = df.drop_duplicates(subset=['north']).reset_index()
    return df

def create_fishnet(xmin, ymin, xmax, ymax, grid_size):
    # Create a list of x and y coordinates for each cell in the fishnet
    y_coords = list(range(int(ymin), int(ymax) + grid_size, grid_size))

    # Create a list of polygons for each cell in the fishnet
    polygons = []
    for y in y_coords:
        polygons.append(Polygon([(xmin, y), (xmax, y), (xmax, y + grid_size), (xmin, y + grid_size)]))
    
    # Create a GeoDataFrame from the list of polygons
    fishnet = gpd.GeoDataFrame(geometry=polygons)
    
    # Set the coordinate reference system (CRS) to WGS84
    fishnet.crs = 'EPSG:4326'
    
    # Add the southern latitude coordinate as an attribute
    fishnet['latitude'] = fishnet.geometry.bounds['miny']
    
    return fishnet

### Soilgrids in GEE community catalog

#### Information
https://gee-community-catalog.org/projects/isric/
* here we are calculating stocks in Gt per 1 degree latitude 

#### Original SOC Stock layer
* 0-30 cm

In [4]:
# Load the SoilGrids dataset for soil organic carbon stocks.
isric_ocs = ee.Image("projects/soilgrids-isric/ocs_mean")

# calculate attribute value multiplied with grid size, scale to km²
# gridcell = t/ha
pixel_area = ee.Image.pixelArea().multiply(1e-4) # grid size in ha
factor = 1e-9 # conversion t to Gt
subset_soc = isric_ocs.multiply(pixel_area).multiply(factor)

### Input Layers
#### BDOD
Bulk density of the fine earth fraction cg/cm³
#### SOC
Soil organic carbon content in the fine earth fraction dg/kg

In [82]:
# Bulk density (cg/cm³)
bdod = ee.Image('projects/soilgrids-isric/bdod_mean')
# Soil organic carbon (dg/kg)
soc = ee.Image('projects/soilgrids-isric/soc_mean')
# Area per Pixel(m²)
pixelArea = ee.Image.pixelArea()

#### replace band names
* images need to have same band names for band calculation

In [6]:
oldNames = bdod.bandNames().getInfo()
replace_string = 'bdod_'
newNames = [n.replace(replace_string, '').replace('_mean', '') for n in oldNames]
bdod_new = bdod.select(oldNames).rename(newNames)

oldNames = soc.bandNames().getInfo()
replace_string = 'soc_'
newNames = [n.replace(replace_string, '').replace('_mean', '') for n in oldNames]
soc_new = soc.select(oldNames).rename(newNames)

#### Create image with layer thickness values as constant

In [7]:
thickness = ee.Image.constant(0.05).rename('0-5cm').addBands(ee.Image.constant(0.1).rename('5-15cm')).addBands(ee.Image.constant(0.15).rename('15-30cm')).addBands(ee.Image.constant(0.3).rename('30-60cm')).addBands(ee.Image.constant(0.4).rename('60-100cm')).addBands(ee.Image.constant(1).rename('100-200cm'))

### Calculation
* soc_pc = "organic carbon percentage of the soil [%]"
* weight = "soil weight per column [kg]"
* soc_stocks = "soil organic carbon carbon stock per column in [t]

In [50]:
soc_pc = soc_new.divide(1000).multiply(bdod_new.divide(100))
weight = bdod_new.multiply(10).multiply(thickness)
soc_stocks = soc_pc.multiply(0.01).multiply(weight).multiply(pixelArea).divide(1000) #t per pixel
pixel_area_extraction = soc_new.mask().multiply(pixelArea).multiply(1e-6).select(['0-5cm'])

### Full global run
* Full global runs will crash !!!
* split into chunks of 10° latitude

### Extract SOC stocks per depth 

In [54]:
# params
size = 10
lat_s = -90
lat_n = 90
image = soc_stocks
# run
df_soc = extract_values_per_latitude(image, lat_s, lat_n, chunk_size=size)
# save
df_soc.to_csv('Total_Carbon_per_zone_global_1degreelat_v2_run3.csv')

  0%|          | 0/18 [00:00<?, ?it/s]

### Extract area 

#### Extract sum of pixel area 

In [55]:
size = 2
lat_s = -90
lat_n = 90
image = pixel_area_extraction
# run
df_area = extract_values_per_latitude(image, lat_s, lat_n, chunk_size=size)
df_area.to_csv('Area_per_zone_global_1degreelat_v3.csv')

  0%|          | 0/90 [00:00<?, ?it/s]

Error, skipped -90
Error, skipped 88
